In [3]:
!pip3 install numpy pandas matplotlib sparqlwrapper

Collecte de données

In [2]:
import requests
import glob
import shutil
import os
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?breed ?breedLabel ?image {
  ?breed wdt:P31 wd:Q12518;
               wdt:P18 ?image.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["breedLabel"]["value"],
            result["image"]["value"],
        )
    )

dataframe = pd.DataFrame(array, columns=["breed", "image"])
dataframe = dataframe.astype(
    dtype={"breed": "<U200", "image": "<U200"}
)



def download_image(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        with open(os.path.join("images", os.path.basename(url)), "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

#dataframe.image.apply(download_image)

Métadonnées des images

In [4]:
import os, sys  
from PIL import Image
from PIL.ExifTags import TAGS

imgfile = Image.open("images/1.jpg")
exif_data = imgfile._getexif()

if exif_data:  # s'il existe des informations EXIF
    for tag, value in exif_data.items():
        if tag in TAGS:
            print(TAGS[tag], value)

GPSInfo {0: b'\x02\x02\x00\x00', 1: 'N', 2: (53.0, 50.3749, 0.0), 3: 'E', 4: (8.0, 38.8513, 0.0), 5: b'\x00', 6: 926.0, 7: (12.0, 32.0, 31.0), 8: '09', 16: 'M', 17: 47.54, 18: 'WGS 84', 29: '2013:09:29'}
ResolutionUnit 2
ExifOffset 291
Make NIKON CORPORATION
Model NIKON D300S
Software Adobe Photoshop Camera Raw 9.12.1 (Windows)
DateTime 2017:10:03 23:02:41
YResolution 300.0
Copyright Â© Ralf Roletschek
XResolution 300.0
Artist Ralf Roletschek
ExifVersion b'0230'
ShutterSpeedValue 7.965784
ApertureValue 6.0
DateTimeOriginal 2013:09:29 13:32:25
DateTimeDigitized 2013:09:29 13:32:25
ExposureBiasValue 0.0
MaxApertureValue 3.0
MeteringMode 2
LightSource 0
Flash 0
FocalLength 200.0
UserComment b'ASCII\x00\x00\x00Foto by Marcela'
ColorSpace 1
DigitalZoomRatio 1.0
FocalLengthIn35mmFilm 300
SceneCaptureType 0
GainControl 0
SubsecTimeOriginal 65
SubsecTimeDigitized 65
Sharpness 0
SubjectDistanceRange 0
SensingMethod 2
FileSource b'\x03'
ExposureTime 0.004
FNumber 8.0
SceneType b'\x01'
ExposurePr

In [19]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS
import glob
from fractions import Fraction
from PIL.TiffTags import TAGS as TIFFTAGS

def get_exif_data(img):
    exif_data = img._getexif()
    exif = {}
    if exif_data:
        for tag, value in exif_data.items():
            if tag in TAGS:
                if isinstance(value, bytes):
                    value = value.decode(errors='ignore')
                elif isinstance(value, (Fraction, int, float)):
                    value = str(value)
                elif isinstance(value, tuple):
                    value = tuple(str(v) for v in value)
                else :
                    value = str(value)
                exif[TAGS[tag]] = value
    return exif

def get_image_metadata(image_path):
    img = Image.open(image_path)
    exif_data = get_exif_data(img)
    exif_data.pop('MakerNote', None)
    metadata = {
        "filename": os.path.basename(image_path),
        "format": img.format,
        "size": img.size,
        "mode": img.mode,
        "orientation": "landscape" if img.width > img.height else "portrait",
        "exif": exif_data
    }
    return metadata

def save_metadata_to_json(metadata, output_file="image_tags.json"):
    try:
        with open(output_file, 'r') as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        data = []

    data.append(metadata)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

image_files = glob.glob("images/*.jpg")
for image_path in image_files:
    metadata = get_image_metadata(image_path)
    save_metadata_to_json(metadata)
    print(f"Métadonnées de l'image {image_path} enregistrées.")

Métadonnées de l'image images/Sender%20Bad%20Mergentheim%20%28Drillberg%29.jpg enregistrées.
Métadonnées de l'image images/Sender%20Bovenden.jpg enregistrées.
Métadonnées de l'image images/Sender%20Vogtsburg-Totenkopf.jpg enregistrées.
Métadonnées de l'image images/Swkrzyzwieza.jpg enregistrées.
Métadonnées de l'image images/Turm%202%20Koenigstuhl%20Heidelberg.jpg enregistrées.
Métadonnées de l'image images/TV-toren%20Goes.jpg enregistrées.
Métadonnées de l'image images/WAK%20THAL%20SCHARFENBURG7.jpg enregistrées.
Métadonnées de l'image images/Waldenburg%20Fernsehturm02%202007-09-22.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Hofkirchen.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Hohe%20Wurzel%20%28Wiesbaden%29.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Kahlheid.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Landau.jpg enregistrées.
Métadonnées de l'image images/Fernmeldeturm%20Lange%20Sicht.jpg enregistrées.